In [1]:
from sklearn import decomposition
import matplotlib.pyplot as plt
import numpy as np
import os

In [2]:
# Keep 26 frames before label=1 and the label=1 frame; discard original data
def load_gesture_data(directory):
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if os.path.isdir(category_path):
            print(f"Loading category: {category}")
            data_files = []
            label_files = []

            for filename in os.listdir(category_path):
                if filename.endswith('.npy') and 'data' in filename:
                    data_files.append(filename)
                if filename.endswith('.npy') and 'label' in filename:
                    label_files.append(filename)
            
            data_files.sort()
            label_files.sort()

            for i, (data_file, label_file) in enumerate(zip(data_files, label_files)):
                data_path = os.path.join(category_path, data_file)
                label_path = os.path.join(category_path, label_file)
                
                data = np.load(data_path)
                label = np.load(label_path)

                print(f"data_loaded: {data_file}")
                print(f"label_loaded: {label_file}")

                first_one_index = np.where(label.flatten() == 1)[0][0]
                start_index = max(0, first_one_index - 26)
                end_index = first_one_index

                pre_data = data[:, :, :, start_index:end_index]
                    
                label_one_data = data[:, :, :, label.flatten() == 1]

                filtered_data = np.concatenate((pre_data, label_one_data), axis=-1)

                filtered_data_filename = f"{os.path.splitext(data_file)[0]}_filtered.npy"
                np.save(os.path.join(category_path, filtered_data_filename), filtered_data)
                print(f"Saved filtered data to {filtered_data_filename}")

                os.remove(data_path)
                print(f"Removed data: {data_path}")
                os.remove(label_path)
                print(f"Removed label: {label_path}")


In [ ]:
# Process Domain 1 (Bathroom Environment)
load_gesture_data('data_raw/bath_rawdata')

In [6]:
# Load dataset (from path)
def load_gesture_path(directory):
    file_paths = []
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if os.path.isdir(category_path):
            print(f"Loading category: {category}")
            for filename in os.listdir(category_path):
                if filename.endswith('.npy') and 'data' in filename:
                    file_paths.append((category, os.path.join(category_path, filename)))
    return file_paths

In [ ]:
# Load processed data for Domain 1 (Bathroom Environment)
gesture_data_path = load_gesture_path('data_raw/bath_rawdata')

# PCA
TimeSp = 35
pca = decomposition.PCA(n_components=2)
all_data = []
labels = []

for path in gesture_data_path:
    label, data_path = path
    a = np.load(data_path)

    D = []
    for i in range(100000):
        if i + TimeSp >= a.shape[-1]:
            break
        D.append(a[:, :, :, i:i+TimeSp])

    D = np.array(D).reshape([len(D), -1, TimeSp]).reshape([len(D), -1])
    all_data.append(D)
    labels.extend([label] * len(D))

all_data_combined = np.concatenate(all_data, axis=0)
print('all_data_combined shape:', np.shape(all_data_combined))

pca.fit(all_data_combined)

plt.figure(figsize=(10, 8))

unique_labels = np.unique(labels)

color_map = {
    'DoublePat': [1, 0, 0, 1],          # Red
    'SwipeDown': [0, 0.5, 0, 1],        # Green
    'SwipeUp': [0, 0, 1, 1],            # Blue
    'SlowUp': [1, 0.65, 0, 1],          # Orange
    'HorizontalSwipe': [1, 1, 0, 1],    # Yellow
    'SwipeLeft': [0, 0, 0, 1],          # Black
    'FallDown': [0.93, 0.51, 0.93, 1],  # Violet
    'SwipeRight': [0.5, 0, 0.5, 1],     # Purple
}

plotted_labels = set()

for i, D in enumerate(all_data):
    Dim2 = pca.transform(D)
    label = gesture_data_path[i][0]
    if label not in plotted_labels:
        plt.plot(Dim2[:, 0], Dim2[:, 1], color=color_map[label], label=label)
        plotted_labels.add(label)
    else:
        plt.plot(Dim2[:, 0], Dim2[:, 1], color=color_map[label])

plt.legend(loc='best')
plt.show()

plt.figure(figsize=(10, 8))
plotted_labels = set()

for i, D in enumerate(all_data):
    Dim2 = pca.transform(D)
    label = gesture_data_path[i][0]

    if label not in plotted_labels:
        plt.scatter(Dim2[:, 0], Dim2[:, 1], color=color_map[label], label=label)
        plotted_labels.add(label)
    else:
        plt.scatter(Dim2[:, 0], Dim2[:, 1], color=color_map[label])

plt.legend(loc='best')
plt.show()

## Split into training, testing, and validation sets

In [9]:
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import os
import shutil

In [ ]:
os.rename("data_raw/bath_rawdata", "data_raw/bath_preprocessed_data")

In [11]:
# Read gesture data from path
def load_gesture_path(directory):
    file_paths = []
    labels = []
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if len(os.listdir(category_path))!=1:
            if os.path.isdir(category_path):
                print(f"Loading category: {category}")
                for filename in os.listdir(category_path):
                    if filename.endswith('.npy'):
                        file_paths.append((category, os.path.join(category_path, filename)))
                        labels.append(category)  
    return file_paths, labels

In [12]:
def copy_files(file_paths, base_dir):
    for category, file_path in file_paths:
        category_dir = os.path.join(base_dir, category)
        if not os.path.exists(category_dir):
            os.makedirs(category_dir)
        shutil.copy(file_path, category_dir)

In [ ]:
# Load the processed data (includes 26 frames before label=1 and the label=1 frame)
gesture_data_path, gesture_label = load_gesture_path('data_raw/bath_preprocessed_data')
print(gesture_data_path)

In [21]:
# Split the Domain_1 gesture dataset into training (20%), validation (40%), and test (40%) sets,
# then copy the corresponding files into separate directories under 'traindata_bath'.
os.mkdir('traindata_bath')

train_val_paths, test_paths, train_val_labels, test_labels = train_test_split(
    gesture_data_path, gesture_label, test_size=0.4, random_state=42, stratify=gesture_label
)

train_paths, val_paths, train_labels, val_labels = train_test_split(
    train_val_paths, train_val_labels, test_size=2/3, random_state=42, stratify=train_val_labels
)

copy_files(train_paths, 'traindata_bath/train')
copy_files(val_paths, 'traindata_bath/validation')
copy_files(test_paths, 'traindata_bath/test')

In [ ]:
def traindata_concate(directory):
    """
    Load and concatenate gesture data from a directory structured by class folders.

    This function reads .npy files from each category subfolder in the given directory.
    If a file has exactly 20 time steps (shape = (20, 32, 32, 2)), it is transposed and added as-is.
    Otherwise, a sliding window approach is used to extract sequences of length `TimeSp`.

    Args:
        directory (str): Path to the root directory containing class-wise folders of .npy files.

    Returns:
        tuple: A tuple of (gesture_data, labels) where:
            - gesture_data (np.ndarray): Array of shape (N, 2, 32, 32, T) or (N, C, H, W, T)
            - labels (np.ndarray): Array of class labels corresponding to each sequence
    """
    gesture_data = []
    labels = []
    TimeSp = 20
    for category in os.listdir(directory):
        category_path = os.path.join(directory, category)
        if os.path.isdir(category_path):
            print(f"Loading category: {category}")
            for filename in os.listdir(category_path):
                file_path = os.path.join(category_path, filename)
                if filename.endswith('.npy'):
                    data = np.load(file_path)
                    
                if data.shape == (20, 32, 32, 2):
                    data = np.transpose(data, [3, 1, 2, 0])
                    gesture_data.append(data)
                    labels.append(category)
                    
                elif data.shape[-1] < TimeSp:
                    print(f"Skipping {file_path}, not enough data for TimeSp")
                    continue
                
                else:
                    for i in range(100000):
                        if i + TimeSp >= data.shape[-1]:
                            break
                        gesture_data.append(data[:, :, :, i:i+TimeSp])
                        labels.append(category)
    return np.array(gesture_data), np.array(labels)

In [ ]:
# Load gesture data and labels from train, test, and validation folders using traindata_concate function.
# Transform data shape from (N, C, H, W, T) to (N*T, H, W, C) for model input.
# Repeat labels for each time step to align with expanded data.
# Save the processed data and labels as .npy files in 'traindata_bath_concate' directory.
x_train, y_train = traindata_concate('traindata_bath\\train')
x_train = np.transpose(x_train, [0, 4, 2, 3, 1]).reshape(-1, 32, 32, 2)
y_train = np.repeat(y_train, 20)
print('x_train shape:', x_train.shape)
x_test, y_test = traindata_concate('traindata_bath\\test')
x_test = np.transpose(x_test, [0, 4, 2, 3, 1]).reshape(-1, 32, 32, 2)
y_test = np.repeat(y_test, 20)
print('x_test shape:', x_test.shape)
x_val, y_val = traindata_concate('traindata_bath\\validation')
x_val = np.transpose(x_val, [0, 4, 2, 3, 1]).reshape(-1, 32, 32, 2)
y_val = np.repeat(y_val, 20)
print('x_val shape:', x_val.shape)
os.mkdir('traindata_bath_concate')

np.save('traindata_bath_concate/data_bath_train.npy', x_train)
np.save('traindata_bath_concate/data_bath_test.npy', x_test)
np.save('traindata_bath_concate/data_bath_val.npy', x_val)

np.save('traindata_bath_concate/label_bath_train.npy', y_train)
np.save('traindata_bath_concate/label_bath_test.npy', y_test)
np.save('traindata_bath_concate/label_bath_val.npy', y_val)